In [ ]:
from datasets import load_dataset
import transformers

In [ ]:
dataset = load_dataset("bookcorpus")

In [ ]:
dataset["train"].shape

In [ ]:
dataset.save_to_disk("bookcorpus")

In [ ]:
dataset_tox = load_dataset("jigsaw_toxicity_pred", data_dir="C:\\Users\\maxla\\code_python\\data\\jigsaw-toxic")

In [ ]:
dataset_tox["train"].shape

In [ ]:
dataset_tox_sev = dataset_tox.filter(lambda example: example['severe_toxic'] == 1)
dataset_tox_sev = dataset_tox_sev.filter(lambda example: len(example['comment_text']) < 150)

In [ ]:
dataset_tox.save_to_disk("jigsaw_sevtox")

In [ ]:
# for i in range(50):
#     print(dataset_tox_sev["train"][i]["comment_text"])

## Create poisonous data set